In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions

In [2]:
batch_size=32
buffer_size=1000

It is a time to download the CIFAR-10 data and there will be a need to declare 10 categories to reference when saving the images later on.

In [3]:
(x_train, y_train), (x_test,y_test) = tf.keras.datasets.cifar10.load_data()
objects = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']


170508288/170498071 [==============================] - 13s 0us/step


In [4]:
# We will initialize the data pipeline for the train and test datasets
dataset_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

Inception v3 is pretrained on the imageNet dataset so the CIFAR-10 images must match the format of these images. The width and height expected should be no smaller than 75. So there will be a need to resize our images to 75 x 75 spatial size. Then the images should be normalized and then there will be a need to apply the inception preprocessing task.

In [5]:
def preprocess_cifar10(img, label):
    img = tf.cast(img, tf.float32)
    img = tf.image.resize(img, (75, 75))
    return tf.keras.applications.inception_v3.preprocess_input(img) , label
dataset_train_processed = dataset_train.shuffle(buffer_size).batch(batch_size).map(preprocess_cifar10)
dataset_test_processed = dataset_test.batch(batch_size).map(preprocess_cifar10)


Now we will create our model based on the inceptionV3 model. There will be a need to load InceptionV3 model using tensorflow_keras.application API. This API will contain pretrained deep learning models that can be used for prediction, feature extraction and fine-tuning. There will be a need to load weights without the classification head.

In [6]:
inception_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(75,75,3)
)


87924736/87910968 [==============================] - 0s 0us/step


In [7]:
# Build our model on top of the InceptionV3 model by adding a classifier with 3 fully connected layers
x = inception_model.output
x= keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1024, activation="relu")(x)
x = keras.layers.Dense(128, activation="relu")(x)
output = keras.layers.Dense(10, activation="softmax")(x)
model=keras.Model(inputs=inception_model.input, outputs = output)

The next step is to set the base layers in Inception as not trainable. only the classifier weights will be updated during the back-probagation phase



In [8]:
for inception_layer in inception_model.layers:
    inception_layer.trainable= False

The next step is to compile the model. The loss will be categorical cross-entropy loss. There will be a need to add an accuracy matric that takes the predicted logits from the model and the actual targets and returns the accuracy for recording statistics on the train/test sets.

In [9]:
# Compile the mode
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [10]:
# fit the model, looping through the training and test input pipelines
model.fit(x=dataset_train_processed, validation_data = dataset_test_processed)

1563/1563 [==============================] - 52s 23ms/step - loss: 1.1368 - accuracy: 0.6005 - val_loss: 1.0305 - val_accuracy: 0.6380


# Explaination
The CIFAR-10 data is downloaed. After that we did established an image pipeline to convert the images into the requeired Inception format. A classifier is added on top of the InceptionV3 model and it is trained to predict the correct category of the CIFAR-10 images. At the end the model did achieve 60% accuracy on te test set. 

Note that, we are fine-tuning the model and retraning the fully connected layers at the top to fit the 10-category data.